<a href="https://colab.research.google.com/github/vijjus/CV/blob/master/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun 13 18:10:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!echo '3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln' > ~/.passwd-s3fs
!cat ~/.passwd-s3fs
!chmod 600 ~/.passwd-s3fs
!mkdir /s3
!apt install s3fs
!s3fs w251-final-project /s3 -o passwd_file=~/.passwd-s3fs,url=https://s3.us.cloud-object-storage.appdomain.cloud
!pip3 install timm

3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln
mkdir: cannot create directory ‘/s3’: File exists
Reading package lists... Done
Building dependency tree       
Reading state information... Done
s3fs is already the newest version (1.82-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
s3fs: MOUNTPOINT directory /s3 is not empty. if you are sure this is safe, can use the 'nonempty' mount option.


In [4]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib
import urllib.request
import cv2
from google.colab.patches import cv2_imshow
import re
import os
import random
from PIL import Image
import timm
from tensorflow import summary
import datetime
%load_ext tensorboard
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
# Retinaface function to extract picture from url

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

# function calls

#url = 'https://github.com/deepinsight/insightface/blob/master/sample-images/t1.jpg?raw=true'
#img = url_to_image(url)
#x = retinaface(img)

# returns the list of information
def retinaface(img):
  model = insightface.app.FaceAnalysis()
  model.prepare(ctx_id = -1, nms = 0.4)
  faces = model.get(img)
  return faces
  """
  for idx, face in enumerate(x):
    print("Face [%d]:"%idx)
    print("\tage:%d"%(face.age))
    gender = 'Male'
    if face.gender==0:
      gender = 'Female'
    print("\tgender:%s"%(gender))
    print("\tembedding shape:%s"%face.embedding.shape)
    print("\tbbox:%s"%(face.bbox.astype(np.int).flatten()))
    print("\tlandmark:%s"%(face.landmark.astype(np.int).flatten()))
    print("")
  """

current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)


In [0]:
phases = ['train', 'val']

In [0]:
#transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [0]:
class DFDC(Dataset):  
  def __init__(self, path, fpv, transform = None):
    self.path = path # relative path to directory of FAKE/REAL videos
    self.index = {} # stores the mapping of indices to video names
    self.mapping = {} # stores the mapping of videos to the number of frames found in the directory for video
    self.boundary = 0 # the index for the boundary between fake and real videos
    self.transform = transform # transform object passed in that is used on the PIL image frames
    i = 0
    regex = "([a-z]+)(\d+)"
    for subdirectory in ["/FAKE", "/REAL"]:
      for x in os.listdir(self.path + subdirectory):
        match = re.search(regex, x)
        if match:
            video = match.group(1)
            number = int(match.group(2))
            if self.mapping.get(video, -1) == -1:
                self.index[i] = video
                i += 1
            self.mapping[video] = max(number, self.mapping.get(video, 0))
      self.boundary = i if self.boundary == 0 else self.boundary
    self.fpv = fpv

  def __getitem__(self, i):
    video = self.index[i]
    num = self.mapping[video]
    x = random.randrange(0, num - self.fpv + 2)
    frames = []
    for k in range(self.fpv):
      subdirectory = "/FAKE" if i < self.boundary else "/REAL"
      frame = Image.open(self.path + subdirectory + "/" + video + str(x)+ ".png")
      frame = frame.convert("RGB")
      if self.transform:
        frame = self.transform(frame)
      frames.append(frame)
      x += 1
    label = 0 if i < self.boundary else 1
    return torch.stack(frames, dim=0), torch.tensor(label, dtype=torch.float32)

  def __len__(self):
    return len(self.index)

In [0]:
input_size = 512
output_size = 1
hidden_dim = 512
n_layers = 2
batchsize = 8
fpv = 2

In [0]:
base_dir = "/s3/DFDC/dataloader/"

In [0]:
def make_dataset(phase):
  d = base_dir + phase
  print(d)
  transform = data_transforms[phase]
  print(transform)
  return DFDC(d, fpv, transform)

In [89]:
dataset = [make_dataset(x) for x in phases]

/s3/DFDC/dataloader/train
Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/s3/DFDC/dataloader/val
Compose(
    Resize(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [90]:
len(dataset[0]), len(dataset[1])

(969, 548)

In [0]:
dataloader = [DataLoader(d, batch_size = batchsize, shuffle = True, num_workers = 2) for d in dataset]

In [92]:
dataloader

In [0]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [0]:
class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        # use mixnet_s, remove the final Linear layer (1536 -> 1000)
        self.mixnet = timm.create_model("mixnet_s", pretrained=True)
        self.mixnet.classifier = Identity()

        self.lstm = nn.LSTM(1536, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc4 = nn.Linear(32, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size, seqlen, c, h, w = x.size()
        x = x.reshape(batch_size*seqlen, c, h, w).float()
        x = self.mixnet(x)
        x = x.reshape(batch_size, seqlen, x.shape[1])
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [0]:
net = DFDCNet(input_size, output_size, hidden_dim, n_layers)

In [0]:
net = net.to(device)

In [65]:
net.train()

DFDCNet(
  (mixnet): EfficientNet(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (1): Sequential(
        (0): InvertedResidual(
          (conv_pw): MixedConv2d(
            (0): Conv2d(8, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(8,

In [0]:
criterion = nn.BCELoss()

In [0]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:
def train():
  for epoch in range(1, 21):
    print("Epoch " + str(epoch), end="")
    for phase in phases:
      if phase == 'train':
        net.train() 
      else:
        net.eval()
      
      dataset_size = 0
      running_loss = 0
      h = net.init_hidden(batchsize)  
 
      # begin processing batch
      for data in dataloader[phases.index(phase)]:
        X, labels = data
        if (len(labels) != batchsize):
          continue
        if phase == 'train':
          net.zero_grad()
        h = tuple([e.data for e in h])
        outputs, h = net(X.to(device), h)
        loss = criterion(outputs, labels.to(device))
        dataset_size += len(labels)
        running_loss += loss.item() * len(labels)
        if phase == 'train':
          loss.backward()
          optimizer.step()
        torch.cuda.empty_cache()
        print(".", end="")

      print("\n")
      print("Epoch " + str(epoch) + " Phase: ", phase, " Loss: ", round(running_loss/dataset_size, 2))

In [0]:
train()

Epoch 1..............................................................................................................